# Project 3

In this project, you will perform a logistic regression on the admissions data we've been working with in projects 1 and 2.

In [84]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
import pylab as pl
import numpy as np


In [85]:
df_raw = pd.read_csv("../assets/admissions.csv")
df = df_raw.dropna() 
df['prestige'] = df['prestige'].astype(int)
print df.head()

   admit    gre   gpa  prestige
0      0  380.0  3.61         3
1      1  660.0  3.67         3
2      1  800.0  4.00         1
3      1  640.0  3.19         4
4      0  520.0  2.93         4


/anaconda/envs/py27_ds_dt_13/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


## Part 1. Frequency Tables

#### 1. Let's create a frequency table of our variables

In [86]:
# frequency table for prestige and whether or not someone was admitted
print 'Frequency Table for prestige'
df['prestige'].value_counts()



Frequency Table for prestige


2    148
3    121
4     67
1     61
Name: prestige, dtype: int64

In [87]:
# frequency table for prestige and whether or not someone was admitted
print 'Frequency Table for admit'
df['admit'].value_counts()

Frequency Table for admit


0    271
1    126
Name: admit, dtype: int64

## Part 2. Return of dummy variables

#### 2.1 Create class or dummy variables for prestige 

In [88]:
prestige_dummies = pd.get_dummies(df['prestige'], prefix='prestige')
#df_new = df.join(pd.get_dummies(df['prestige'], prefix='prestige', drop_first=True))

#prestige_dummies
#df_new

#features = [i for i in prestige_dummies.columns if 'prestige_' in i]
#features += ['gre']
#features += ['gpa']
#features += ['admit']
#features



#### 2.2 When modeling our class variables, how many do we need? 



Answer: We need one less than the count of unique values in the variable. In this case for 'prestige' , we need 3 class variables.

## Part 3. Hand calculating odds ratios

Develop your intuition about expected outcomes by hand calculating odds ratios.

In [89]:
cols_to_keep = ['admit', 'gre', 'gpa']
handCalc = df[cols_to_keep].join(prestige_dummies.ix[:, 'prestige_1':])
print handCalc.head()

   admit    gre   gpa  prestige_1  prestige_2  prestige_3  prestige_4
0      0  380.0  3.61           0           0           1           0
1      1  660.0  3.67           0           0           1           0
2      1  800.0  4.00           1           0           0           0
3      1  640.0  3.19           0           0           0           1
4      0  520.0  2.93           0           0           0           1


In [91]:
#crosstab prestige 1 admission 
# frequency table cutting prestige and whether or not someone was admitted

print pd.crosstab(handCalc['prestige_1'],handCalc['admit'])


admit         0   1
prestige_1         
0           243  93
1            28  33


#### 3.1 Use the cross tab above to calculate the odds of being admitted to grad school if you attended a #1 ranked college

Odds of being admitted if you attend a 1 ranked college = 33/(28+33) = 0.54

#### 3.2 Now calculate the odds of admission if you did not attend a #1 ranked college

Odds of being admitted if you did not attend a 1 ranked college = 93/(243+93) = 0.28

#### 3.3 Calculate the odds ratio

In [ ]:
0.54/0.28 = 1.93  ## check if this is correct################

#### 3.4 Write this finding in a sentenance: 

Answer: The odds of being admitted to a grad school if one attended #1 ranked college is 1.93.
probability of admission = 66% 

#### 3.5 Print the cross tab for prestige_4

In [92]:
print pd.crosstab(handCalc['prestige_4'],handCalc['admit'])

admit         0    1
prestige_4          
0           216  114
1            55   12


#### 3.6 Calculate the OR 

In [ ]:
Odds of being admitted if you attended a 4 ranked college = 12/(12+55) = 0.18
Odds of being admitted if you did not attend a 4 ranked college = 114/(114+216) = 0.35

Odds Ratio (OR) = 0.18/0.35 = 0.51 ## check if this is correct################

#### 3.7 Write this finding in a sentence

Answer: If a student attended a #4 ranked grad school , the odds (likelihood) of getting admitted to a grad school is 0.51. The probability of getting admission is ( 0.51/1+0.51 ) = 32%. 

## Part 4. Analysis

In [93]:
# create a clean data frame for the regression
cols_to_keep = ['admit', 'gre', 'gpa']
data = df[cols_to_keep].join(prestige_dummies.ix[:, 'prestige_2':])
print data.head()

   admit    gre   gpa  prestige_2  prestige_3  prestige_4
0      0  380.0  3.61           0           1           0
1      1  660.0  3.67           0           1           0
2      1  800.0  4.00           0           0           0
3      1  640.0  3.19           0           0           1
4      0  520.0  2.93           0           0           1


We're going to add a constant term for our Logistic Regression. The statsmodels function we're going to be using requires that intercepts/constants are specified explicitly.

In [94]:
# manually add the intercept
data['intercept'] = 1.0

#### 4.1 Set the covariates to a variable called train_cols

In [95]:
train_cols = data.columns[1:]
train_cols

Index([u'gre', u'gpa', u'prestige_2', u'prestige_3', u'prestige_4',
       u'intercept'],
      dtype='object')

#### 4.2 Fit the model

In [96]:
logit = sm.Logit(data['admit'], data[train_cols])
result = logit.fit()

Optimization terminated successfully.
         Current function value: 0.573854
         Iterations 6


#### 4.3 Print the summary results

In [68]:
print result.summary()

                           Logit Regression Results                           
Dep. Variable:                  admit   No. Observations:                  397
Model:                          Logit   Df Residuals:                      391
Method:                           MLE   Df Model:                            5
Date:                Tue, 06 Jun 2017   Pseudo R-squ.:                 0.08166
Time:                        00:28:18   Log-Likelihood:                -227.82
converged:                       True   LL-Null:                       -248.08
                                        LLR p-value:                 1.176e-07
                   coef    std err          z      P>|z|      [95.0% Conf. Int.]
--------------------------------------------------------------------------------
gre              0.0022      0.001      2.028      0.043      7.44e-05     0.004
gpa              0.7793      0.333      2.344      0.019         0.128     1.431
prestige_2.0    -0.6801      0.317     -2.14

#### 4.4 Calculate the odds ratios of the coeffiencents and their 95% CI intervals

hint 1: np.exp(X)

hint 2: conf['OR'] = params
        
           conf.columns = ['2.5%', '97.5%', 'OR'] 

In [97]:
print np.exp(result.params)  ## Odds ratios of the co-efficients

gre           1.002221
gpa           2.180027
prestige_2    0.506548
prestige_3    0.262192
prestige_4    0.211525
intercept     0.020716
dtype: float64


In [98]:
#print result.conf_int()
params = result.params
conf = result.conf_int()
conf['OR'] = params
conf.columns = ['2.5%', '97.5%', 'OR']
print np.exp(conf)

                2.5%     97.5%        OR
gre         1.000074  1.004372  1.002221
gpa         1.136120  4.183113  2.180027
prestige_2  0.272168  0.942767  0.506548
prestige_3  0.133377  0.515419  0.262192
prestige_4  0.093329  0.479411  0.211525
intercept   0.002207  0.194440  0.020716


#### 4.5 Interpret the OR of Prestige_2

Answer: ??? What does Odds Ratio imply for a Categorical Variable. Does this help us calculate the probability of getting an admission if the student attended a Rank 2 College?

#### 4.6 Interpret the OR of GPA

Answer: A unit increase in gpa increases the probablility of getting an admission by 118%.

## Part 5: Predicted probablities


As a way of evaluating our classifier, we're going to recreate the dataset with every logical combination of input values. This will allow us to see how the predicted probability of admission increases/decreases across different variables. First we're going to generate the combinations using a helper function called cartesian (above).

We're going to use np.linspace to create a range of values for "gre" and "gpa". This creates a range of linearly spaced values from a specified min and maximum value--in our case just the min/max observed values.

In [99]:
def cartesian(arrays, out=None):
    """
    Generate a cartesian product of input arrays.
    Parameters
    ----------
    arrays : list of array-like
        1-D arrays to form the cartesian product of.
    out : ndarray
        Array to place the cartesian product in.
    Returns
    -------
    out : ndarray
        2-D array of shape (M, len(arrays)) containing cartesian products
        formed of input arrays.
    Examples
    --------
    >>> cartesian(([1, 2, 3], [4, 5], [6, 7]))
    array([[1, 4, 6],
           [1, 4, 7],
           [1, 5, 6],
           [1, 5, 7],
           [2, 4, 6],
           [2, 4, 7],
           [2, 5, 6],
           [2, 5, 7],
           [3, 4, 6],
           [3, 4, 7],
           [3, 5, 6],
           [3, 5, 7]])
    """

    arrays = [np.asarray(x) for x in arrays]
    dtype = arrays[0].dtype

    n = np.prod([x.size for x in arrays])
    if out is None:
        out = np.zeros([n, len(arrays)], dtype=dtype)

    m = n / arrays[0].size
    out[:,0] = np.repeat(arrays[0], m)
    if arrays[1:]:
        cartesian(arrays[1:], out=out[0:m,1:])
        for j in xrange(1, arrays[0].size):
            out[j*m:(j+1)*m,1:] = out[0:m,1:]
    return out

In [134]:
# instead of generating all possible values of GRE and GPA, we're going
# to use an evenly spaced range of 10 values from the min to the max 
gres = np.linspace(data['gre'].min(), data['gre'].max(), 10)
print gres
# array([ 220.        ,  284.44444444,  348.88888889,  413.33333333,
#         477.77777778,  542.22222222,  606.66666667,  671.11111111,
#         735.55555556,  800.        ])
gpas = np.linspace(data['gpa'].min(), data['gpa'].max(), 10)
print gpas
# array([ 2.26      ,  2.45333333,  2.64666667,  2.84      ,  3.03333333,
#         3.22666667,  3.42      ,  3.61333333,  3.80666667,  4.        ])


# enumerate all possibilities
combos = pd.DataFrame(cartesian([gres, gpas, [1, 2, 3, 4], [1.]]))

[ 220.          284.44444444  348.88888889  413.33333333  477.77777778
  542.22222222  606.66666667  671.11111111  735.55555556  800.        ]
[ 2.26        2.45333333  2.64666667  2.84        3.03333333  3.22666667
  3.42        3.61333333  3.80666667  4.        ]


#### 5.1 Recreate the dummy variables

In [108]:
# recreate the dummy variables
combos.columns = ['gre', 'gpa', 'prestige', 'intercept']
prestige_dummies = pd.get_dummies(combos['prestige'], prefix='prestige')
prestige_dummies
prestige_dummies.columns = ['prestige_1', 'prestige_2', 'prestige_3', 'prestige_4']

# keep only what we need for making predictions
cols_to_keep = ['gre', 'gpa', 'prestige', 'intercept']
combos = combos[cols_to_keep].join(prestige_dummies.ix[:, 'prestige_2':])
combos

,gre,gpa,prestige,intercept,prestige_2,prestige_3,prestige_4
0,220.0,2.260000,1.0,1.0,0,0,0
1,220.0,2.260000,2.0,1.0,1,0,0
2,220.0,2.260000,3.0,1.0,0,1,0
3,220.0,2.260000,4.0,1.0,0,0,1
4,220.0,2.453333,1.0,1.0,0,0,0
5,220.0,2.453333,2.0,1.0,1,0,0
6,220.0,2.453333,3.0,1.0,0,1,0
7,220.0,2.453333,4.0,1.0,0,0,1
8,220.0,2.646667,1.0,1.0,0,0,0
9,220.0,2.646667,2.0,1.0,1,0,0


#### 5.2 Make predictions on the enumerated dataset

In [111]:
combos['admit_pred'] = result.predict(combos[train_cols])
print combos.tail()

       gre       gpa  prestige  intercept  prestige_2  prestige_3  prestige_4  \
395  800.0  3.806667       4.0        1.0           0           0           1   
396  800.0  4.000000       1.0        1.0           0           0           0   
397  800.0  4.000000       2.0        1.0           1           0           0   
398  800.0  4.000000       3.0        1.0           0           1           0   
399  800.0  4.000000       4.0        1.0           0           0           1   

     admit_pred  
395    0.334286  
396    0.734040  
397    0.582995  
398    0.419833  
399    0.368608  


#### 5.3 Interpret findings for the last 4 observations

Answer: The last four observations ( last 4 students ) have the same gre and gpa and have a different rank( prestige). Given that four students have the same gre and gpa , the rank of their undergrad college determines their probability of getting an admission. Students with more prestigious colleges ( Ordered as Rank 1 - 4 ) have a higher probablity of getting admission given the other factors are same. 

## Bonus

Plot the probability of being admitted into graduate school, stratified by GPA and GRE score.